In [1]:
import pandas as pd

a_col = ['anime_id', 'name','members']
animes = pd.read_csv('data/anime.csv', sep=',', usecols=a_col)

ratings = pd.read_csv('data/ratings_clean_1.csv', sep=',')


ratings.head()

,user_id,anime_id,rating
0,7,22,7
1,7,30,10
2,7,31,9
3,7,32,9
4,7,59,7


In [2]:
userRatings = ratings.pivot_table(index='user_id',columns='anime_id',values='rating')

userRatings.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,34107,34238,34239,34240,34252,34283,34324,34325,34367,34475
user_id,,,,,,,,,,,,,,,,,,,,,
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,9.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
corrMatrix = userRatings.corr(method='pearson', min_periods=10)
corrMatrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,34107,34238,34239,34240,34252,34283,34324,34325,34367,34475
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.545609,0.304228,0.184979,0.020803,0.099349,0.271919,0.169293,0.165864,0.276748,...,NaN,NaN,NaN,0.168130,NaN,NaN,NaN,NaN,NaN,NaN
5,0.545609,1.000000,0.318142,0.274115,0.076639,-0.018466,0.229941,0.203949,0.238644,0.180292,...,NaN,NaN,NaN,0.124070,NaN,NaN,NaN,NaN,NaN,NaN
6,0.304228,0.318142,1.000000,0.285032,0.149235,0.287464,0.126820,0.153482,0.279346,0.202733,...,NaN,NaN,NaN,0.255565,NaN,NaN,NaN,NaN,NaN,NaN
7,0.184979,0.274115,0.285032,1.000000,0.058391,0.348318,0.198889,0.381646,0.157672,0.084932,...,NaN,NaN,NaN,0.073845,NaN,NaN,NaN,NaN,NaN,NaN
8,0.020803,0.076639,0.149235,0.058391,1.000000,0.371133,0.011783,0.459859,0.342006,-0.218541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(corrMatrix.shape)
mis_reviews = pd.DataFrame({
    'user_id': [999],  
    'anime_id': [1089],  
    'rating': [10]  
})
mis_reviews.to_csv('data/my_ratings.csv', index=False)
mis_ratings = pd.merge(ratings, mis_reviews[['anime_id', 'rating']])
myRatings = pd.Series(data=mis_ratings['rating'].values, index=mis_ratings['anime_id'])


(8962, 8962)


In [5]:
simCandidates = pd.Series(dtype=float)

for anime, rating in myRatings.items():
    if anime not in corrMatrix.columns:
        continue  # ignora si el anime no está en la matriz
    sims = corrMatrix[anime].dropna()
    sims = sims.map(lambda x: x * rating)
    simCandidates = pd.concat([simCandidates, sims])

simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates = simCandidates.drop(myRatings.index, errors='ignore')
simCandidates = simCandidates.sort_values(ascending=False)


    
simdf = simCandidates.reset_index()
simdf.columns = ['anime_id','score']
recomendaciones = pd.merge(simdf, animes, on='anime_id', how='left')
recomendaciones.sort_values('score', ascending=False, inplace=True)
print(recomendaciones[['name', 'score']].head(10))

                              name       score
0       Boku no Chikyuu wo Mamotte  482.808027
1                 Teito Monogatari  469.475113
2          Yuru Yuri Nachuyachumi!  465.070423
3  Manie-Manie: Meikyuu Monogatari  461.373855
4      Gall Force 1: Eternal Story  457.668538
5              Mamono Hunter Youko  444.977658
6           Casshern: Robot Hunter  444.686945
7                   Robot Carnival  439.984606
8           Senkaiden Houshin Engi  439.673980
9              Hakuouki Reimeiroku  421.932022
